In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import soundfile as sf
from pathlib import Path
import re
import json
import os
import sys
sys.path.append('..\soundbay')
from utils.metadata_processing import (reorder_columns_to_default_view, 
                                       correct_call_times_with_duration,
                                       non_overlap_df, bg_from_non_overlap_calls)

In [15]:
sys.path.append('C:/Users')
selection_path = Path('C:/Users/amitg/Documents/Deep Voice/HF WAV Manatee Samples/BB_analysis/2017-Jul-08-12-67407878-RestingHole1A')
#recording_path = Path('/mnt/c/users/amitg/Documents/Deep Voice/HF WAV Manatee Samples/170708155527')
selection_path2 = Path('C:/Users/amitg/Documents/Deep Voice/HF WAV Manatee Samples/BB_analysis/2017-Jul-14-17-67407878-RestingHole1A')

filelist = list(selection_path.glob('*.txt'))
filelist2 = list(selection_path2.glob('*.txt'))

filelist_short = []
for file in filelist:
    name_ext = os.path.basename(file)
    name, txt = os.path.splitext(name_ext)
    name = re.search(r'67\d{6,6}.\d{12,12}', file.as_posix()).group() 
    filelist_short.append(name)

In [33]:
filelist2

[WindowsPath('C:/Users/amitg/Documents/Deep Voice/HF WAV Manatee Samples/BB_analysis/2017-Jul-14-17-67407878-RestingHole1A/67407878.170713215715.Table.1.selections copy_bb.txt'),
 WindowsPath('C:/Users/amitg/Documents/Deep Voice/HF WAV Manatee Samples/BB_analysis/2017-Jul-14-17-67407878-RestingHole1A/67407878.170713222715.Table.1.selections cop_bby.txt'),
 WindowsPath('C:/Users/amitg/Documents/Deep Voice/HF WAV Manatee Samples/BB_analysis/2017-Jul-14-17-67407878-RestingHole1A/67407878.170713225715.Table.1.selections copy-BB.txt'),
 WindowsPath('C:/Users/amitg/Documents/Deep Voice/HF WAV Manatee Samples/BB_analysis/2017-Jul-14-17-67407878-RestingHole1A/67407878.170714002715.Table.1.selections copy-BB.txt'),
 WindowsPath('C:/Users/amitg/Documents/Deep Voice/HF WAV Manatee Samples/BB_analysis/2017-Jul-14-17-67407878-RestingHole1A/67407878.170714025715.Table.1.selections copy-BB.txt'),
 WindowsPath('C:/Users/amitg/Documents/Deep Voice/HF WAV Manatee Samples/BB_analysis/2017-Jul-14-17-67407

In [17]:
textfile = open("ManateesList2.txt","w")
for f in filelist_short:
    textfile.write(f+"\n")
textfile.close()

In [47]:
metadata=[]
metadata2 = []
for file in filelist:
    dfTemp = pd.read_csv(file, sep="\t")
    dfTemp['filename'] = re.search(r'17\d{10,10}', file.as_posix()).group() 
    dfTemp['StartMicInWater'] =np.amin(dfTemp['Begin Time (s)'])
    dfTemp['EndMicInWater'] =np.amax(dfTemp['End Time (s)'])
    dfTemp['filepath'] = os.path.basename(selection_path)#os.path.join(,file,'.txt')
    metadata.append(dfTemp)
for file in filelist2:
    dfTemp = pd.read_csv(file, sep="\t")
    dfTemp['filename'] = re.search(r'17\d{10,10}', file.as_posix()).group() 
    dfTemp['StartMicInWater'] =np.amin(dfTemp['Begin Time (s)'])
    dfTemp['EndMicInWater'] =np.amax(dfTemp['End Time (s)'])
    dfTemp['filepath'] = os.path.basename(selection_path2)#os.path.join(,file,'.txt')
    metadata2.append(dfTemp)

metadata = pd.concat(metadata)
metadata.rename(columns={'Begin Time (s)': 'begin_time', 'End Time (s)': 'end_time'}, inplace=True)
metadata2 = pd.concat(metadata2)
metadata2.rename(columns={'Begin Time (s)': 'begin_time', 'End Time (s)': 'end_time'}, inplace=True)
frames = [metadata,metadata2]
PositiveCalls = pd.concat(frames, axis=0)
PositiveCalls = PositiveCalls.drop(['Type of NLP','UpdatedNLP','SNR code','manatee','Manatee', 'MANATEE VOC',
                                    'mANATEE','2ndMeasure','AB Notes','AB notes'], axis=1)
PositiveCalls['label'] = np.ones((len(PositiveCalls),), dtype=int) #assuming everything is calls. TODO: filter out by notes


In [48]:
PositiveCalls

,Selection,View,Channel,begin_time,end_time,Low Freq (Hz),High Freq (Hz),Center Time (s),Delta Freq (Hz),Delta Time (s),...,Type,No. harm,Notes,filename,StartMicInWater,EndMicInWater,filepath,>22kHz,MANATEE,label
0,1,Spectrogram 1,1,1031.939,1032.106,6047.0,12289.0,1032.021,6242.0,0.167,...,NaN,NaN,NaN,170708215525,1031.939,4277.935,2017-Jul-08-12-67407878-RestingHole1A,NaN,NaN,1
1,2,Spectrogram 1,1,1049.332,1049.529,6827.2,12191.5,1049.429,5364.3,0.197,...,NaN,NaN,NaN,170708215525,1031.939,4277.935,2017-Jul-08-12-67407878-RestingHole1A,NaN,NaN,1
2,3,Spectrogram 1,1,1053.876,1054.018,7412.4,11703.8,1053.997,4291.4,0.142,...,NaN,NaN,NaN,170708215525,1031.939,4277.935,2017-Jul-08-12-67407878-RestingHole1A,NaN,NaN,1
3,4,Spectrogram 1,1,1058.840,1059.004,7314.9,11898.9,1058.910,4584.0,0.164,...,NaN,NaN,NaN,170708215525,1031.939,4277.935,2017-Jul-08-12-67407878-RestingHole1A,NaN,NaN,1
4,5,Spectrogram 1,1,1081.540,1081.655,7997.6,10923.6,1081.591,2926.0,0.115,...,NaN,NaN,NaN,170708215525,1031.939,4277.935,2017-Jul-08-12-67407878-RestingHole1A,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8,9,Spectrogram 1,1,79.719,80.150,0.0,2656.0,79.941,2656.0,0.431,...,NaN,NaN,NaN,170717062706,28.071,139.716,2017-Jul-14-17-67407878-RestingHole1A,NaN,N,1
9,10,Spectrogram 1,1,82.962,83.008,1448.7,15590.8,82.980,14142.1,0.046,...,NaN,NaN,NaN,170717062706,28.071,139.716,2017-Jul-14-17-67407878-RestingHole1A,NaN,N,1
10,11,Spectrogram 1,1,83.268,83.268,6898.6,6898.6,83.267,0.0,0.000,...,NaN,NaN,NaN,170717062706,28.071,139.716,2017-Jul-14-17-67407878-RestingHole1A,NaN,N,1
11,13,Spectrogram 1,1,134.872,135.122,3441.6,22548.3,135.029,19106.7,0.250,...,NaN,NaN,NaN,170717062706,28.071,139.716,2017-Jul-14-17-67407878-RestingHole1A,NaN,Y,1


In [49]:
PositiveCalls = non_overlap_df(PositiveCalls)
annotation_file = bg_from_non_overlap_calls(PositiveCalls)
annotation_file.to_csv('Manatee_annotations_bg_from_calls.csv', index=False)

../soundbay\utils\metadata_processing.py:113: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  file_df.sort_values(by='begin_time', inplace=True)
